In [1]:
from glob import glob
import json

In [2]:
files = glob('texts.jsonl*.splitted.requested')
files

['texts.jsonl05.splitted.requested',
 'texts.jsonl02.splitted.requested',
 'texts.jsonl06.splitted.requested',
 'texts.jsonl04.splitted.requested',
 'texts.jsonl03.splitted.requested',
 'texts.jsonl01.splitted.requested',
 'texts.jsonl07.splitted.requested',
 'texts.jsonl00.splitted.requested']

In [3]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
                
            mapping[l['src']] = l['r']['result']
            
len(mapping)

701888

In [4]:
all_data = []
with open('train_9.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data = []
        for d_ in l['data']:
            data.append({
                'value': d_,
                'value_ms': mapping.get(d_)
            })
        all_data.append(data)

In [5]:
all_data[0]

[{'value': 'Could you paraphrase and summarize the given text material? Answer according to: They cut you really deep.\nThe place where anything can occur.\nWhat right do they have to tell me what to do?\nOnes they can’t even see.\nFor they are what they are.\nIntelligence comes with deep processing power.\nSo there is good amongst the bad.\nFor the way that I am.\nSomeone to live with, sleep with and be intimate with.\nAnd my need for peace and quiet.\nAnd fall when I’m sad.\nIt’s the only chance we’ll get.',
  'value_ms': 'Bolehkah anda menghuraikan dan meringkaskan bahan teks yang diberikan? Jawapan mengikut: Mereka memotong anda dengan sangat dalam.\nTempat di mana apa-apa boleh berlaku.\nApa hak mereka untuk memberitahu saya apa yang perlu dilakukan?\nYang mereka tidak dapat lihat.\nKerana mereka adalah apa adanya.\nKepintaran datang dengan kuasa pemprosesan yang mendalam.\nJadi ada kebaikan di antara yang buruk.\nUntuk cara saya.\nSeseorang untuk tinggal bersama, tidur dengan dan

In [6]:
break_at = [
    'help.openai.com',
]

In [16]:
from tqdm import tqdm
import random

all_texts = []
for i in tqdm(range(len(all_data))):
    texts = []
    for no, d in enumerate(all_data[i]):
        
        if no % 2 == 0:
            t = '<manusia>: '
            if random.random() > 0.5:
                c = 'value'
            else:
                c = 'value_ms'
        else:
            t = '<bot>: '
                
        if d[c] is None:
            break
        
        if any([b in d[c].lower() for b in break_at]):
            print(c[c], '\n-----\n')
            break
        
        n = d[c]
        n = n.strip()
        if len(n) < 3:
            break
        t = t + n
        texts.append(t)

    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|███████████████████████████████| 118349/118349 [00:01<00:00, 113918.92it/s]


In [17]:
len(all_texts)

114361

In [18]:
all_texts[0]

"<manusia>: Bolehkah anda menghuraikan dan meringkaskan bahan teks yang diberikan? Jawapan mengikut: Mereka memotong anda dengan sangat dalam.\nTempat di mana apa-apa boleh berlaku.\nApa hak mereka untuk memberitahu saya apa yang perlu dilakukan?\nYang mereka tidak dapat lihat.\nKerana mereka adalah apa adanya.\nKepintaran datang dengan kuasa pemprosesan yang mendalam.\nJadi ada kebaikan di antara yang buruk.\nUntuk cara saya.\nSeseorang untuk tinggal bersama, tidur dengan dan menjadi intim.\nDan keperluan saya untuk kedamaian dan ketenangan.\nDan jatuh apabila saya sedih.\nIa satu-satunya peluang yang kita akan dapat.\n<bot>: Parafrasa: Lokasi tidak dapat diramalkan dan ada pihak yang cuba mengawal tindakan kita walaupun kurang pemahaman. Kecerdasan memerlukan usaha kognitif yang ketara dan terdapat kedua-dua sifat positif dan negatif dalam individu. Keinginan untuk persahabatan dan ketenangan, serta mengalami emosi yang rendah, adalah semua keperluan semula jadi manusia. Kita harus m

In [19]:
for t in all_texts:
    splitted = t.split('<manusia>:')
    if '<bot>:' not in splitted[-1]:
        print(t)
        break

In [20]:
with open('part9-alternate_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')